In [1]:
# -*- coding: utf-8 -*-
import sys
import os
import unicodedata
import string
import urllib2

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

# Agrego al path la carpeta modulos
sys.path.insert(0, os.path.abspath("../../modulos"))
from LexAnalyser import LexAnalyser

In [2]:
url = 'http://www.gutenberg.org/cache/epub/2000/pg2000.txt'
response = urllib2.urlopen(url)
raw = response.read()
start = raw.find('El ingenioso hidalgo don Quijote de la Mancha')
end = raw.rfind('End of Project Gutenberg')
raw = raw[start:end]
print len(raw)

2178990


In [3]:
vocabulario = {}
la = LexAnalyser({})
terms = la.analyse(raw.decode("UTF-8"))["terms"]
for t in terms:
    if t not in vocabulario:
        vocabulario[t] = 0
    vocabulario[t] += 1
print "Vocabulario (extracto)", [{key: val} for key, val in vocabulario.items()[:10]]
print "Cantidad de terminos: ", len(terms)

Vocabulario (extracto) [{u'consuelome': 1}, {u'canes': 3}, {u'igual': 16}, {u'indignacion': 2}, {u'hermana': 11}, {u'hermano': 106}, {u'gastara': 1}, {u'acababamos': 1}, {u'dictado': 1}, {u'compuesta': 11}]
Cantidad de terminos:  381215


In [4]:
df = pd.DataFrame(vocabulario.items(), 
                  columns = ['Termino','Frecuencia'])
df.set_index('Termino', inplace = True)

df['Ranking'] = df['Frecuencia'].rank(method = 'min', ascending = False)
df['C=F*R'] = df['Frecuencia'] * df['Ranking']

df.sort_values(['Frecuencia'], ascending=False, inplace=True)

df.head()

,Frecuencia,Ranking,C=F*R
Termino,,,
que,21475,1.0,21475.0
de,18298,2.0,36596.0
y,18188,3.0,54564.0
la,10363,4.0,41452.0
a,9823,5.0,49115.0


In [5]:
# Agrego alguna data extra 
n = df['Frecuencia'].sum()
df['Frec. Acum.'] = df['Frecuencia'].cumsum()
df['Frec. Rel.'] = df['Frecuencia'] / n
df['Frec. Rel. Acum.'] = df['Frec. Acum.'] / n
df.head()

,Frecuencia,Ranking,C=F*R,Frec. Acum.,Frec. Rel.,Frec. Rel. Acum.
Termino,,,,,,
que,21475,1.0,21475.0,21475,0.056333,0.056333
de,18298,2.0,36596.0,39773,0.047999,0.104332
y,18188,3.0,54564.0,57961,0.047711,0.152043
la,10363,4.0,41452.0,68324,0.027184,0.179227
a,9823,5.0,49115.0,78147,0.025768,0.204995


In [6]:
# Selecciono solo aquella data que tiene frecuencia < 5
df_menor_5 = df[(df['Frecuencia'] < 5.0)]
df_menor_5.to_csv('extras/quijote_frecuencia_menor_5.csv')
df_menor_5.head()

,Frecuencia,Ranking,C=F*R,Frec. Acum.,Frec. Rel.,Frec. Rel. Acum.
Termino,,,,,,
item,4,5270.0,21080.0,353749,0.00001,0.927951
importunidades,4,5270.0,21080.0,353753,0.00001,0.927962
valentisimo,4,5270.0,21080.0,353757,0.00001,0.927972
ijadas,4,5270.0,21080.0,353761,0.00001,0.927983
obrar,4,5270.0,21080.0,353765,0.00001,0.927993


In [11]:
porcentaje_omitido = (1 - df_menor_5["Frec. Rel. Acum."].min()) * 100
print "Quitando del índice aquellos terminos cuya frecuencia es menor que 5 se omite el %.2f por ciento del total de términos" % porcentaje_omitido

Quitando del índice aquellos terminos cuya frecuencia es menor que 5 se omite el 7.20 por ciento del total de términos


In [92]:
# PROPORCIÓN SEGUN ZIPF

logx = np.log(df["Ranking"])
logy = np.log(df["Frecuencia"])
rectaAjuste = np.polyfit(logx, logy, 1)
pendiente = rectaAjuste[0]
ordenadaAlOrigen = rectaAjuste[1]

frecuenciaDada = 5.0
logFrec = np.log(frecuenciaDada)

# A partir de la ecuación
# log_frecuencia = pendiente * logx + ordenadaAlOrigen
# se tiene log_frecuencia y se busca averiguar el valor de logx
# por lo tanto:
rankLog = (logFrec - ordenadaAlOrigen) / pendiente
rank = np.exp(rankLog)

df_zipf = df.loc[df["Ranking"] > rank]
df_zipf.head()

,Frecuencia,Ranking,C=F*R,Frec. Acum.,Frec. Rel.,Frec. Rel. Acum.
Termino,,,,,,
estiende,6,3969.0,23814.0,346706,0.000016,0.909476
soberbios,6,3969.0,23814.0,346712,0.000016,0.909492
juzgue,6,3969.0,23814.0,346718,0.000016,0.909508
seca,6,3969.0,23814.0,346724,0.000016,0.909523
vengamos,6,3969.0,23814.0,346730,0.000016,0.909539


In [91]:
porcentaje_omitido = (1 - df_zipf["Frec. Rel. Acum."].min()) * 100
print "Segun Zipf se omite el %.2f por ciento del total de términos" % porcentaje_omitido

Segun Zipf se omite el 9.05 por ciento del total de términos


In [97]:
# EXTRA - PRUEBA DE OBTENER FRECUENCIA A PARTIR DE RANKING
# PROPORCION SEGUN ZIPF
ranking = 5270
rectaAjuste = np.polyfit(np.log(df["Ranking"]), np.log(df["Frecuencia"]), 1)
rectaEvaluada = np.poly1d(rectaAjuste)

frecuenciaEstimada = np.exp(rectaEvaluada(np.log(ranking)))

print "Frecuencia estimada: ", frecuenciaEstimada
print "Frecuencia real: ", df[df['Ranking'] == 5270]["Frecuencia"].mean()

Frecuencia estimada:  3.34640794496
Frecuencia real:  4.0
